In [ ]:
import param
import paramnb

# Tooltips

Doc strings supplied when a parameter is declared are displayed as a 'tooltip' when the mouse hovers over the widget label (``x`` in the widget below)

In [ ]:
class TooltipExample(param.Parameterized):
    x = param.Number(default=1,bounds=(0,2),doc="X position")

In [ ]:
paramnb.Widgets(TooltipExample)

# Recursive editing

Parameter types such as ``param.ObjectSelector`` allow selection of another Parameterized object; paramNB also allows the parameters of the selected object to be edited by clicking the **...** button next to any selected Parameterized object:

In [ ]:
class Location(param.Parameterized):
    duration = param.Integer(default=5, bounds=(0,10))

locations = [Location(name="One"),Location(name="Two")]

class Employee(param.Parameterized):
    age = param.Integer(default=30,bounds=(18,100))
    location = param.ObjectSelector(default=locations[0],objects=locations)

class Task(param.Parameterized):
    employee = param.ObjectSelector(objects=[Employee(name="A"),Employee(name="B")])

In [ ]:
paramnb.Widgets(Task)

In [ ]:
Task.employee.location.duration

# View parameters

The paramNB library provides an easy way to manipulate parameters on ``Parameterized`` using the widgets in the notebook. In addition to controlling input parameters a common usecase for using widgets in the notebook is to dynamically control some visual display output. In addition to all the standard parameters supplied by the ``param`` library, ``paramNB`` also supplies so called ``View`` parameters, which render their output in a widget area. The output parameters may be updated simply by setting the parameter on the class.

In the first simple example we will declare a Parameterized class with a ``Number`` parameter called magnitude and an ``HTML`` parameter which will let us display some arbitrary HTML. In this case we will simply generate a pandas dataframe with random data within the update method and use the ``to_html`` method to convert it to an HTML table. If we define the ``update`` method as the callback of the widgets the table will now update whenever the slider is dragged. To ensure that the output is drawn on initialization we set ``on_init=True``.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class HTMLExample(param.Parameterized):
    
    magnitude = param.Number(1, bounds=(0, 10))

    output = paramnb.view.HTML()
    
    def update(self, **kwargs):
        self.output = pd.DataFrame(np.random.rand(10,2)*self.magnitude).to_html()

example = HTMLExample(name='HTMLExample')
paramnb.Widgets(example, on_init=True, callback=example.update)

The ``HTML`` parameter accepts any arbitrary HTML string but for convenience paramNB also allows supplying a custom ``renderer`` function, which converts the view data to HTML. In this case we declare to parameters to control the ``amplitude`` and ``frequency`` of a sine curve and then declare an ``HTML`` parameter which uses a HoloViews MPLRenderer to render the output. Note that we can additionally supply the size of the output as a tuple of ``(width, height)`` in pixels, in this case (300, 300).

Additionally we can declare the ``view_position``, which specifies where the viewing widget will be placed in relation to the input widgets:

In [ ]:
import holoviews as hv
renderer = hv.renderer('matplotlib')

class ImageExample(param.Parameterized):

    color = param.Color(default='#000000', precedence=0)

    element = param.ObjectSelector(default=hv.Curve,
                                   objects=[hv.Curve, hv.Scatter, hv.Area],
                                   precedence=0)

    amplitude = param.Number(default=2, bounds=(2, 5))
    
    frequency = param.Number(default=2, bounds=(1, 10))
    
    output = paramnb.view.Image(renderer=lambda x: (renderer(x)[0], (300, 300)))

    def update(self, **kwargs):
        self.output = self.element(self.amplitude*np.sin(np.linspace(0, np.pi*self.frequency)),
                                  vdims=[hv.Dimension('y', range=(-5, 5))])(style=dict(color=self.color))

example = ImageExample(name='HoloViews Example')
paramnb.Widgets(example, callback=example.update, on_init=True, view_position='right')

Finally, the generic ``View`` parameter may also be used to display the rich ``repr`` of any object, effectively mirroring the output of IPython's ``display`` function. If we load the HoloViews extension with the bokeh backend, and subclass the ImageExample, we can also render bokeh plots in this way:

In [ ]:
hv.extension('bokeh')

class ViewExample(ImageExample):
    
    output = paramnb.view.View()
    
example = ViewExample(name='HoloViews+Bokeh Example')
paramnb.Widgets(example, callback=example.update, on_init=True, view_position='right')